In [1]:
import os

import cv2
import layoutparser as lp
import numpy as np
import pandas as pd
import pytesseract

from pdf2image import convert_from_path

In [2]:
os.mkdir('../data/processed/test')

In [3]:
pdf_path = "../data/raw/test.pdf"
images = convert_from_path(pdf_path, dpi=300)

In [4]:
model = lp.models.Detectron2LayoutModel(
    "../models/config.yml",
    "../models/model_final.pth",
    extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8],
    label_map={0: "Text", 1: "Title", 2: "List", 3: "Table", 4: "Figure"},
)

In [5]:
for i, image in enumerate(images):
    image_cv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    layout = model.detect(image_cv)

    tables = [b for b in layout if b.type == "Table"]

    for t_idx, table_block in enumerate(tables):
        segment = table_block.crop_image(image_cv)

        # --- Step 3: OCR the table image ---
        ocr_text = pytesseract.image_to_string(segment, config="--psm 6")

        # --- Step 4: Convert OCR to DataFrame ---
        lines = [line.split() for line in ocr_text.split("\n") if line.strip()]
        df = pd.DataFrame(lines)

        # --- Step 5: Save to Excel ---
        excel_path = f"../data/processed/test/output_page{i+1}_table{t_idx+1}.xlsx"
        df.to_excel(excel_path, index=False, header=False)


/Users/davidhackett12/projects/historical_city_populations/venv/lib/python3.12/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:4316.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
